## Current best
- Without cabin
- Discretize Age on 5 categories and distance=0.25
- Discretize Fare on 5 categories and distance=0.15

In [ ]:
import pandas as pd
import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from discretizer import Discretizer
from lem2 import *

In [ ]:
test_data = pd.read_csv("./test.csv").drop(["Name", "Ticket", "Cabin"], axis=1)
train_data = pd.read_csv("./train.csv").drop(['PassengerId', "Name", "Ticket", "Cabin"], axis=1)

print(f"train_data.shape: {train_data.shape}")
print(f"test_data.shape: {test_data.shape}")

In [ ]:
disc = Discretizer()
disc.fit(train_data, ['Age'], number_of_output_values=5, distance_from_extreme_values=.25)
disc.fit(train_data, ['Fare'], number_of_output_values=4, distance_from_extreme_values=.2, verbose=1)

train_data = disc.discretize(train_data)
test_data = disc.discretize(test_data)

print(train_data['Survived'].value_counts())

In [ ]:
preds = pd.DataFrame({})

for column in train_data.drop("Survived", axis=1).columns:
    print(f"Model without {column}:")
    lem2 = LEM2()
    lem2.fit(train_data.drop(["Survived", column], axis=1), train_data["Survived"], only_certain=False, verbose=1)
    lem2.evaluate(train_data.drop(["Survived", column], axis=1), train_data["Survived"])
    
    preds[f"without {column}"] = lem2.predict(test_data)
    
print(f"Model with all columns:")
lem2 = LEM2()
lem2.fit(train_data.drop(["Survived"], axis=1), train_data["Survived"], only_certain=False, verbose=1)
lem2.evaluate(train_data.drop(["Survived"], axis=1), train_data["Survived"])
preds[f"all"] = lem2.predict(test_data)

In [ ]:
preds

def get_mode(row):
    return row.mode().iloc[0]
    
mv = preds.apply(lambda row: get_mode(row), axis=1)

preds["mode"] = mv
preds

test_data["Survived"] = preds["mode"]

In [ ]:
# test_data["Survived"] = lem2.predict(test_data)
test_data["Survived"] = test_data["Survived"].astype(int)
test_data[["PassengerId", "Survived"]].to_csv("preds.csv", index=False)